In [133]:
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
import gzip
import io
import json

# Initialize the S3 client
s3 = boto3.client('s3')

# Specify the bucket name and object key
bucket_name = 'vcthackathonbucket1'
object_key = 'game-changers/games/2022/val:0429689c-85fa-4448-bbfb-10a099864809.json.gz'

In [134]:
def getUnzippedData(bucket_name, object_key):
    try:
        # Retrieve the object
        response = s3.get_object(Bucket=bucket_name, Key=object_key)
    
        # Read the object's content into memory
        data = response['Body'].read()
    
        # Unzip the gzipped content
        with gzip.GzipFile(fileobj=io.BytesIO(data)) as gz:
            unzipped_data = (gz.read())
    
        return json.loads(unzipped_data)
    
    except NoCredentialsError:
        print("Credentials not available.")
    except PartialCredentialsError:
        print("Incomplete credentials provided.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [135]:
# A cohesive dictionary which will contain all the information we want to feed into the LLM 
players = {}

PARSE LEAGUES

In [136]:
# Specify the bucket name and object key
bucket_name = 'vcthackathon-data'
object_key = 'game-changers/esports-data/leagues.json.gz'

leagues = {}
leaguesJSON = getUnzippedData(bucket_name, object_key)

In [137]:
league = {}
#print(leaguesJSON[:2])
for league in leaguesJSON:
    leagues[league['league_id']] = {'name': league['name'], 'region': league['region']}

print(len(leagues))

11


PARSE TEAMS

In [138]:
# Specify the bucket name and object key
bucket_name = 'vcthackathon-data'
object_key = 'game-changers/esports-data/teams.json.gz'

teams = {}
teamsJSON = getUnzippedData(bucket_name, object_key)
for team in teamsJSON:
        teams[team['id']] = {'name': team['name'], 'acronym': team['acronym'], 'home_league_id': team['home_league_id']}
print(len(teams))

283


PARSE TOURNAMENTS

In [139]:
# Specify the bucket name and object key
bucket_name = 'vcthackathon-data'
object_key = 'game-changers/esports-data/tournaments.json.gz'

tournaments = {}
tournamentsJSON = getUnzippedData(bucket_name, object_key)
for tournament in tournamentsJSON:
    tournaments[tournament['id']] = {'league_id': tournament['league_id'], 'name': tournament['name']}

print(len(tournaments))

57


PARSE PLAYERS

In [140]:
from datetime import datetime

bucket_name = 'vcthackathon-data'
object_key = 'game-changers/esports-data/players.json.gz'

players = {}
playersJSON = getUnzippedData(bucket_name, object_key)

date_object = datetime.strptime(playersJSON[0]['updated_at'], "%Y-%m-%dT%H:%M:%SZ")
# print(f'# of players in JSON: {len(playersJSON)}')
for player in playersJSON:
    currentDate = datetime.strptime(player['updated_at'], "%Y-%m-%dT%H:%M:%SZ")
    if player['id'] in players:

        #if a duplicate entry, keep the most recent update
        if currentDate > players[player['id']]['updated_at']:
            
            # if most recent is archived, remove them
            if player['status']=="archived":
                # if most recent status of a player is archived
                del players[player['id']]
                
            #otherwise, update them
            else:
                players[player['id']]['handle'] = player['handle']
                players[player['id']]['updated_at'] = currentDate
                players[player['id']]['first_name'] = player['first_name']
                players[player['id']]['last_name'] = player['last_name']
                players[player['id']]['home_team_id'] = player['home_team_id']
                
    else:
        if player['status'] == 'archived':
            continue
        players[player['id']] = {}
        players[player['id']]['handle'] = player['handle']
        players[player['id']]['updated_at'] = currentDate
        players[player['id']]['first_name'] = player['first_name']
        players[player['id']]['last_name'] = player['last_name']
        players[player['id']]['home_team_id'] = player['home_team_id']

print(len(players))

1498


PARSE MAPPING DATA
* note that mapping data is per game *

In [141]:
mappingData = {}

bucket_name = 'vcthackathon-data'
object_key = 'game-changers/esports-data/mapping_data_v2.json.gz'

mappingDataJSON = getUnzippedData(bucket_name, object_key)
for map in mappingDataJSON:
    mappingData[map['platformGameId']] = {
        'esportsGameId': map['esportsGameId'],
        #'matchId': map['matchId'],
        'tournamentId': map['tournamentId'],
        'teamMapping': map['teamMapping'],
        'participantMapping': map['participantMapping']
    }
    
print(len(mappingData))

1140


In [142]:
# Load the agent mapping (this should be done outside of the config_event function and loaded once)
import json 

agent_code_mapping = {}
with open('agentCodeMapping.json', 'r') as f:
    agent_code_mapping = json.load(f)

print(len(agent_code_mapping))

23


In [206]:
def config_event(event, temp_players, mapping_info):
    platform_game_id = event['platformGameId']
        
    # Extract map info
    map_info = event['configuration']['selectedMap']['fallback']['displayName']
    
    
    # Extract team and player information
    for team in event['configuration']['teams']:
        team_id = str(team['teamId']['value'])
        team_name = mapping_info['teamMapping'].get(team_id, "Unknown Team")     
        
        for player in team['playersInTeam']:
            player_id = player['value']
            
            # Find player info from the players list in the configuration event
            player_info = next((p for p in event['configuration']['players'] if p['playerId']['value'] == player_id), None)
            if not player_info:
                continue  # Skip if player info is not found
        
            
            # Extract player details
            player_handle = player_info['accountId']['value']
            player_name = player_info['displayName']
            selected_agent_guid = player_info['selectedAgent']['fallback']['guid']  # GUID of the agent
            
            # Get the agent name from the GUID using agentCodeMapping
            agent_name = agent_code_mapping.get(selected_agent_guid, "Unknown Agent")
            
            # Get global player ID from participantMapping
            global_player_id = mapping_info['participantMapping'].get(str(player_id))
            if not global_player_id:
                continue  # Skip if no global player ID is found
            
            # Initialize the player entry if not already present
            if global_player_id not in temp_players:
                temp_players[global_player_id] = {
                    'ID': player_handle,
                    'handle': player_handle,
                    'name': player_name,
                    'team': team_name, 
                    'Games': []
                }
                # Add game-specific information
                game_info = {
                    'Tournament Name': mapping_info['tournamentId'],
                    'Attacking Kills': 0,
                    'Defending Kills': 0,
                    'Attacking Deaths': 0,
                    'Defending Deaths': 0,
                    'Assists (attacking/defending)': 0,
                    'Map': map_info,
                    'Agent': agent_name  
                }
            
                temp_players[global_player_id]['Games'].append(game_info)
    
    return temp_players


In [174]:
def player_died_event(event, temp_players, mapping_info, attack_defend_info):
        
    # Extract player IDs
    deceased_id = event['playerDied']['deceasedId']['value']
    killer_id = event['playerDied']['killerId']['value']
    
    # Use participantMapping to map to global player ID
    
    global_deceased_id = mapping_info['participantMapping'].get(str(deceased_id))
    global_killer_id = mapping_info['participantMapping'].get(str(killer_id))
    
    # print('global id is', global_deceased_id)
    # for keys, values in temp_players.items():
    #     print(keys)
    
    # If no valid mappings are found, return
    if not global_deceased_id or not global_killer_id:
        return temp_players
    
    # Determine if the deceased and killer were on attacking or defending teams
    if attack_defend_info['attackingTeam'] == mapping_info['participantMapping'].get(str(deceased_id)):
        temp_players[global_deceased_id]['Games'][-1]['Attacking Deaths'] += 1
    else:
        temp_players[global_deceased_id]['Games'][-1]['Defending Deaths'] += 1
    
    if attack_defend_info['attackingTeam'] == mapping_info['participantMapping'].get(str(killer_id)):
        temp_players[global_killer_id]['Games'][-1]['Attacking Kills'] += 1
    else:
        temp_players[global_killer_id]['Games'][-1]['Defending Kills'] += 1

In [160]:
# here we set/count/increment the rounds won for the winning team members
def round_decided(event, attack_defend_info, temp_players, mapping_info):
    platform_game_id = event['platformGameId']
    
    # Get the winning team ID
    winning_team = event['roundDecided']['result']['winningTeam']['value']
    
    # Iterate over players in the winning team and update rounds won
    for local_player_id, global_player_id in mapping_info['participantMapping'].items():
        if local_player_id in temp_players:
            current_team = temp_players[global_player_id]['team']
            if current_team == winning_team:
                if current_team == attack_defend_info['attackingTeam']:
                    temp_players[global_player_id]['Games'][-1]['roundsWon']['attack'] += 1
                else:
                    temp_players[global_player_id]['Games'][-1]['roundsWon']['defense'] += 1
    
    return temp_players


In [161]:
def round_started(event, attack_defend_info):
    round_started = event['roundStarted']
    
    # Update attacking and defending teams
    attack_defend_info['attackingTeam'] = round_started['spikeMode']['attackingTeam']['value']
    attack_defend_info['defendingTeam'] = round_started['spikeMode']['defendingTeam']['value']
    return attack_defend_info


In [216]:
# this function takes in a game data file, and parses through it, returning only what we want
# @returns updated players dictionary 
def parse_game_data(gameData, players, mappingData):
    
    # check if the game is in the mapping data. if not, skip it.
    platform_game_id = gameData[0]['platformGameId']
    if platform_game_id not in mappingData:
        print(f"Platform game ID {platform_game_id} not found in mapping data")
        return players
    # else:
        # print(f"Platform game value {mappingData[platform_game_id]}")
    
    
    temp_players = {}
    attackDefendInfo = {}
    mapping_info = mappingData[platform_game_id]
    configRan = False
    for event in gameData:
        if 'configuration' in event:
            if configRan == False:
                config_event(event, temp_players, mapping_info)
                configRan = True # only need to run config once per game 
        elif 'playerDied' in event:
            player_died_event(event, temp_players, mapping_info, attackDefendInfo)
        elif 'roundStarted' in event:
            round_started(event, attackDefendInfo)
        elif 'roundDecided' in event:
            round_decided(event, attackDefendInfo, temp_players, mapping_info)
        elif 'roundStarted' in event:
            round_started(event, attackDefendInfo)
            
    
    print("temp_players", (temp_players))
    # 
    # for keys,values in temp_players.items():
    #     print(keys, values)
    #     print()
    
    # Merge temp_players into the main players dictionary
    for player_id, player_data in temp_players.items():
        if player_id in players:
            # Merge game information if player already exists in the players dictionary
            
            # Ensure the 'Games' key exists for the player
            if 'Games' not in players[player_id]:
                players[player_id]['Games'] = []  # Initialize if it doesn't exist

            # Merge game information
            players[player_id]['Games'].extend(player_data['Games'])
            
        else:
            # Add new player entry if not in players dictionary
            players[player_id] = player_data
            
    
    # return players

    

In [148]:
# TODO: Change this to be all game files instead of just one
bucket_name = 'vcthackathon-data'
object_key = 'game-changers/games/2024/val:001949ff-e8a0-40fb-8ba5-65e123df8e1d.json.gz'

gameData = getUnzippedData(bucket_name, object_key)

In [149]:
parse_game_data(gameData, players, mappingData)
print(len(players))

Platform game value {'esportsGameId': '112729035488859981', 'tournamentId': '112729035485517625', 'teamMapping': {'12': '108764409368519858', '13': '108748169745284534'}, 'participantMapping': {'1': '111067958377934403', '2': '108748099637064824', '3': '111057371424123924', '4': '108748093144321973', '5': '108748093245509945', '6': '109030070859531583', '7': '108764516777284603', '8': '108748092607804521', '9': '108764513807403457', '10': '110745687059117485'}}
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
val:001949ff-e8a0-40fb-8ba5-65e123df8e1d
v

In [150]:
bucket_name = 'vcthackathon-data'
object_key = 'game-changers/games/2024/val:3d07d83d-909c-40c7-85fb-62fef2d6b9e2.json.gz'

gameData = getUnzippedData(bucket_name, object_key)

In [151]:

eventList = {}
for event in gameData:
    keys, values = zip(*event.items())
    # events.add(keys[1])
    eventList[keys[1]]=eventList.get(keys[1], []) + [values]
    # eventList[keys[1]] = values


print(len(eventList))
    
    

keys, values = zip(*eventList.items())
for key in keys:
    print(key)
    
    print(eventList[key][1])
    
    print()


25
snapshot
('val:3d07d83d-909c-40c7-85fb-62fef2d6b9e2', {'observers': [{'displayName': 'GCL Archie', 'observerId': {'value': 11}, 'followedPlayerId': {'value': -1}, 'tagLine': 'eProd'}, {'displayName': 'GCL Observer3', 'observerId': {'value': 12}, 'followedPlayerId': {'value': -1}, 'tagLine': 'eProd'}, {'displayName': 'GCL Observer1', 'observerId': {'value': 13}, 'followedPlayerId': {'value': -1}, 'tagLine': 'eProd'}, {'displayName': 'GCL Observer2', 'observerId': {'value': 14}, 'followedPlayerId': {'value': -1}, 'tagLine': 'eProd'}, {'displayName': 'GCL Safi', 'observerId': {'value': 15}, 'followedPlayerId': {'value': -1}, 'tagLine': 'eProd'}, {'displayName': 'GCL Rabanito', 'observerId': {'value': 16}, 'followedPlayerId': {'value': -1}, 'tagLine': 'eProd'}, {'displayName': '', 'observerId': {'value': 22}, 'followedPlayerId': {'value': -1}, 'tagLine': ''}, {'displayName': '', 'observerId': {'value': 23}, 'followedPlayerId': {'value': -1}, 'tagLine': ''}, {'displayName': '', 'observer

IndexError: list index out of range

In [152]:
for event in eventList['roundCeremony']:
    print(event)

('val:3d07d83d-909c-40c7-85fb-62fef2d6b9e2', {'fallback': {'guid': '', 'displayName': 'DefaultCeremony_C'}, 'type': 'DEFAULT'}, {'serverInfo': {'processId': '', 'rfc190Scope': 'aresriot.aws-mia1-prod.tournament-gp-miami-1.ares.gameserver'}, 'playback': 0, 'sequenceNumber': 903, 'stage': 1, 'gameVersion': '09.02.00.2703179', 'gameId': {'value': '3d07d83d-909c-40c7-85fb-62fef2d6b9e2'}, 'eventTime': {'includedPauses': '585.664s', 'omittingPauses': '585.664s'}, 'wallTime': '2024-08-05T23:02:43.940Z', 'currentGamePhase': {'phase': 'ROUND_ENDING', 'roundNumber': 0}})
('val:3d07d83d-909c-40c7-85fb-62fef2d6b9e2', {'fallback': {'guid': '', 'displayName': 'DefaultCeremony_C'}, 'type': 'DEFAULT'}, {'serverInfo': {'processId': '', 'rfc190Scope': 'aresriot.aws-mia1-prod.tournament-gp-miami-1.ares.gameserver'}, 'playback': 0, 'sequenceNumber': 1452, 'stage': 1, 'gameVersion': '09.02.00.2703179', 'gameId': {'value': '3d07d83d-909c-40c7-85fb-62fef2d6b9e2'}, 'eventTime': {'includedPauses': '800.590s', 

In [218]:
parse_game_data(gameData, players, mappingData)

temp_players {'107600796790972557': {'ID': 'e73a7280-6103-5c9d-8d6e-4354042983ff', 'handle': 'e73a7280-6103-5c9d-8d6e-4354042983ff', 'name': 'AKV mmonch', 'team': '107600865871683810', 'Games': [{'Tournament Name': '112710290866353527', 'Attacking Kills': 0, 'Defending Kills': 15, 'Attacking Deaths': 0, 'Defending Deaths': 17, 'Assists (attacking/defending)': 0, 'Map': 'Infinity', 'Agent': 'Unknown Agent'}]}, '107600825476014259': {'ID': '23ce7a87-9a9c-5f77-8c35-7fa7cc6ffec4', 'handle': '23ce7a87-9a9c-5f77-8c35-7fa7cc6ffec4', 'name': 'AKV Ivy', 'team': '107600865871683810', 'Games': [{'Tournament Name': '112710290866353527', 'Attacking Kills': 0, 'Defending Kills': 15, 'Attacking Deaths': 0, 'Defending Deaths': 16, 'Assists (attacking/defending)': 0, 'Map': 'Infinity', 'Agent': 'Unknown Agent'}]}, '107600792569471113': {'ID': 'e8dd0405-e24a-5853-bae4-3a8264321d47', 'handle': 'e8dd0405-e24a-5853-bae4-3a8264321d47', 'name': 'AKV LiNLiN', 'team': '107600865871683810', 'Games': [{'Tourname

In [217]:
keys, values = zip(*mappingData.items())
print(mappingData[keys[0]])

{'esportsGameId': '110614988238486652', 'tournamentId': '110614878961138673', 'teamMapping': {'18': '107185970379663407', '19': '108240763447862128'}, 'participantMapping': {'1': '110196391536301033', '2': '108550500823868280', '3': '107691403279337920', '4': '108231581673551401', '5': '107282823220699313', '6': '107025879465378213', '7': '107025879556684669', '8': '107186009704881408', '9': '107186009230387255', '10': '110615094848109973'}}
